## Calculate the interface energies with PFP and LightPFP

* The interface energy will be calculate with the following formula

$$
E_{interface} = E_{Ni/TiO_2} - E_{Ni} - E_{TiO_2}
$$

* We take the last frame of MD trajectories (at 1200K) as the initial interface structures.

In [ ]:
model_version="v7.0.0"
calc_mode="crystal_u0"

model_id = "" # SMALL Model
workdir = "light_pfp_md_small"

# model_id = "" # LARGE Model
# workdir = "light_pfp_md_large"

pfp_dir = "pfp_md"

In [ ]:
from light_pfp_client import ASECalculator as LASECalculator 
from light_pfp_client import Estimator as LEstimator
from pfp_api_client import Estimator, ASECalculator

lpfp_calc = LASECalculator(LEstimator(model_id = model_id))
calc = ASECalculator(Estimator(model_version=model_version, calc_mode=calc_mode))

In [ ]:
from ase.io import read
from ase.optimize.fire import FIRE
from ase.filters import FrechetCellFilter
from IPython.display import clear_output


def opt(atoms):
    fcf = FrechetCellFilter(atoms)
    fire = FIRE(fcf)
    fire.run(steps = 1000, fmax=0.1)
    clear_output()

In [ ]:
from mp_api.client import MPRester

api_key = "" # Your Materials Project API KEY

with MPRester(api_key) as m:
    anatase_tio2 = m.get_structure_by_material_id("mp-390", conventional_unit_cell=True)
    ni = m.get_structure_by_material_id("mp-23", conventional_unit_cell=True)

## 1. Get the potential energy of crystal Ni and TiO2

In [ ]:
import numpy as np

anatase_tio2_pfp = anatase_tio2.copy()
anatase_tio2_pfp.calc = calc
opt(anatase_tio2_pfp)
E_tio2_pfp = anatase_tio2_pfp.get_potential_energy() / (np.sum(anatase_tio2_pfp.numbers==22))
print(f"Energy of TiO2, PFP: {E_tio2_pfp} eV")

In [ ]:
anatase_tio2_lpfp = anatase_tio2.copy()
anatase_tio2_lpfp.calc = lpfp_calc
opt(anatase_tio2_lpfp)
E_tio2_lpfp = anatase_tio2_lpfp.get_potential_energy() / (np.sum(anatase_tio2_lpfp.numbers==22))
print(f"Energy of TiO2, PFP: {E_tio2_lpfp} eV")

In [ ]:
ni_pfp = ni.copy()
ni_pfp.calc = calc
opt(ni_pfp)
E_ni_pfp = ni_pfp.get_potential_energy() / (np.sum(ni_pfp.numbers==28))
print(f"Energy of Ni, PFP: {E_ni_pfp} eV")

In [ ]:
ni_lpfp = ni.copy()
ni_lpfp.calc = lpfp_calc
opt(ni_lpfp)
E_ni_lpfp = ni_lpfp.get_potential_energy() / (np.sum(ni_lpfp.numbers==28))
print(f"Energy of Ni, PFP: {E_ni_lpfp} eV")

## 2. Get the energy of interface structures

In [ ]:
from ase.io import Trajectory

maters = ["anatase_TiO2_Ni_0", "anatase_TiO2_Ni_1", "anatase_TiO2_Ni_2", "anatase_TiO2_Ni_3", "anatase_TiO2_Ni_4", "anatase_TiO2_Ni_5"]
title = [
    "Ni (111)/TiO2 (101) (Ti termination)",
    "Ni (111)/TiO2 (101) (O termination)",
    "Ni (110)/TiO2 (001) (Ti termination)",
    "Ni (110)/TiO2 (001) (O termination)",
    "Ni (100)/TiO2 (001) (Ti termination)",
    "Ni (100)/TiO2 (001) (O termination)",
]
surface_energy_lpfp = []
surface_energy_pfp = []


for mater in maters:
    # LPFP
    atoms = Trajectory(f"{workdir}/light_pfp_md_{mater}_1200.traj")[-1]
    atoms.calc = lpfp_calc
    opt(atoms)
    E_lpfp = atoms.get_potential_energy()
    n_ni = np.sum(atoms.numbers == 28)
    n_tio2 = np.sum(atoms.numbers == 22)
    surf = np.linalg.norm(np.cross(atoms.cell[0], atoms.cell[1]))
    Es_lpfp = (E_lpfp - n_ni * E_ni_lpfp - n_tio2 * E_tio2_lpfp) / surf
    surface_energy_lpfp.append(Es_lpfp)
    # PFP
    atoms = Trajectory(f"{pfp_dir}/pfp_md_{mater}_1200.traj")[-1]
    atoms.calc = calc
    opt(atoms)
    E_pfp = atoms.get_potential_energy()
    n_ni = np.sum(atoms.numbers == 28)
    n_tio2 = np.sum(atoms.numbers == 22)
    surf = np.linalg.norm(np.cross(atoms.cell[0], atoms.cell[1]))
    Es_pfp = (E_pfp - n_ni * E_ni_pfp - n_tio2 * E_tio2_pfp) / surf
    surface_energy_pfp.append(Es_pfp)

## 3. Plot the results

In [ ]:
import matplotlib.pyplot as plt


# Set the positions and width for the bars
x = np.arange(len(maters))
width = 0.35

# Create the bar plot
fig, ax = plt.subplots(figsize=(8, 8))
bar1 = ax.bar(x - width/2, surface_energy_lpfp, width, label='LPFP', color='skyblue')
bar2 = ax.bar(x + width/2, surface_energy_pfp, width, label='PFP', color='orange')

# Add labels, title, and legend
ax.set_ylabel('Interface Energy')
ax.set_xticks(x)
ax.set_xticklabels(title, rotation=60.0)
ax.legend()

# Optionally, add data labels
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(bar1)
add_labels(bar2)

# Tight layout to ensure labels are not cut off
fig.tight_layout()

# Show plot
plt.savefig("interface_energy.png")